In [1]:
!git clone https://github.com/aakashvardhan/custom-lit-yolov3

Cloning into 'custom-lit-yolov3'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 120 (delta 26), reused 106 (delta 12), pack-reused 0
Receiving objects: 100% (120/120), 2.42 MiB | 25.30 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [2]:
%cd custom-lit-yolov3

/content/custom-lit-yolov3


In [3]:

%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('/content/custom-lit-yolov3')

In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 30.3 MB/s eta 0:00:00


In [6]:
import torch
from lit_model import YOLOv3LightningModule
from lit_data_module import YOLODataModule
from pytorch_lightning import Trainer
import lightning as pl
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor, RichModelSummary,EarlyStopping
import torch
import os
import config as cfg

In [7]:
torch.cuda.empty_cache()
pl.seed_everything(1, workers=True)

INFO: Seed set to 1
INFO:lightning.fabric.utilities.seed:Seed set to 1


1

In [8]:
train_dir = '/content/custom-lit-yolov3/dataset/PASCAL_VOC/train.csv'
test_dir = '/content/custom-lit-yolov3/dataset/PASCAL_VOC/test.csv'
datamodule = YOLODataModule(config=cfg, train_dir=train_dir, test_dir=test_dir)
datamodule.setup()

In [9]:
model = YOLOv3LightningModule(config=cfg)

In [17]:
from callbacks import ClassAccuracyCallback, PlotExampleCallback, MAPCallback

In [18]:
EPOCHS = int(cfg.NUM_EPOCHS * 0.4)

In [23]:
trainer = pl.Trainer(precision=16, max_epochs=EPOCHS,
                              callbacks=[ModelCheckpoint(dirpath="checkpoints/",verbose=True,),
                              ClassAccuracyCallback(config=cfg,train_n_epochs=1, test_n_epochs=10),
                              PlotExampleCallback(config=cfg, n_epochs = EPOCHS),
                              MAPCallback(config=cfg, test_n_epochs = EPOCHS),
                              RichModelSummary(max_depth=-1),
                              LearningRateMonitor()])

/usr/local/lib/python3.10/dist-packages/lightning/fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
